# Explore Simple Neural Networks with PyTorch
Copyright 2021, LEAKY.AI LLC

In this exercise, we will build some very simple neural networks using PyTorch and explore their properties.
- Use Google Colab to develop simple neural networks using PyTorch
- Understand how to construct neural networks and estimate their size

To get started, head over to Google Colab via this link:

https://colab.research.google.com

Then load the GitHub project using the following URL:
[FILL OUT] https://github.com/LeakyAI/BirdDetector.git

Good luck!

## Step 1 - Import the PyTorch Libraries
PyTorch has several libraries we will need to build this project.  The main library is the torch library.  We will also load the torchvision library which contains the pre-trained neural network we will need for our project as well as some transformation libraries that will help us process the image before passing it to our neural network.


In [28]:
# Import PyTorch, PyTorch transform module and our pre-trained RESNET50 network
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

## Step 2 - Build a Simple Model
Here you will build a simple neural network

In [32]:
# Build a transformation for each image passed into our network
class MySimpleNetwork(nn.Module):
    
        # Network takes 2 inputs, produces 1 output
        def __init__(self):
            super(MySimpleNetwork,self).__init__()
            self.fc1 = nn.Linear(2,1)
            
        def forward(self, x):
            out = self.fc1(x)
            yHat = F.relu(out)
            return yHat
    
# Create an instance of the model and print out summary
net = MySimpleNetwork()
net = net.cuda()
summary(net,(1,2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]               3
Total params: 3
Trainable params: 3
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


## Step 3 - Build a Multi-Layer Model


In [34]:
# Build a transformation for each image passed into our network
class MySimpleNetwork(nn.Module):
    
        # Network takes 2 inputs, produces 1 output
        def __init__(self):
            super(MySimpleNetwork,self).__init__()
            self.fc1 = nn.Linear(2,10)
            self.fc2 = nn.Linear(10,1)
        
        def forward(self, x):
            outFC1 = F.relu(self.fc1(x))
            outFC2 = F.relu(self.fc2(outFC1))
            return outFC2

# Create an instance of the model and print out summary
net = MySimpleNetwork()
net = net.cuda()
summary(net,(1,2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 10]              30
            Linear-2                 [-1, 1, 1]              11
Total params: 41
Trainable params: 41
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


## Step 4 - Pass the Image Through our Network
Next, we will pass the image into our nerual network.  Before doing so, we will pre-process the image with our transform.  This will ensure the input image meets the input requirements of the neural network regardless of it's original size.

## Step 5 - Convert Outputs to Meaningful Categories
Each of the outputs of the RESNET model is associated with a specific object (car, flower etc.).  We will need the names of each of these classes which we can find on the pytorch hub list:

https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

Let's read them into a list so we can clearly determine the class the network is predicting.

In [ ]:
# From PyTorch Documentation - read in all ImageNet class names
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
# Show top categories per image
top3_prob, top3_catid = torch.topk(probabilities, 3)

# Print out the values
for i in range(top3_prob.size(0)):
    print(f"{categories[top3_catid[i]]} ({top3_prob[i].item():.3%}) {top3_catid[i]}")

## Step 6 - Detect Birds
There are 1000 classes predicted by the RESNET50 model.  These all come from ImageNet and you have been provided the list along with the class ids in a pdf which you can download.  Looking at the full list of classes, you will see three seperate sections for birds.  The bird class IDs include 7..24, 80..100 and 127..146.  Hence, if your model detects with high confidence one of these classes, it would be a good assumption that a bird would be present in the picture. 

In [ ]:
# Define a function to check if a class ID is a type of bird
# Input id, returns True or False
def BirdClassID(id):
    if (7<=id<=24) or (80<=id<=100) or (127<=id<=146):
        return True
    else:
        return False

In [ ]:
# Function that takes an image as input and determines if a 
# bird is present in the image
def BirdDetector(im):
    
    # Turn off gradient calculations as we are only using inference here
    with torch.no_grad():
        
        # Transform Input and add a batch dimension to the tensor
        inp = myTransform(im).unsqueeze(0)
        
        # Pass the input through the RESNET model
        out = net(inp)

    # Convert the output to probabilities
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    
    # Grap the single highest probability and associated id
    prob, catid = torch.topk(probabilities, 1)

    # If a bird is detected, print out "Bird Detected", otherwise "No bird"
    if BirdClassID(catid):
        print ("Bird Detected!")
    else:
        print ("No bird...")
    
    # Output the predicted name and probability
    print(f"Predicted {categories[catid]} with {prob.item():.3%} probability.")

## Try with your own image!

## Key Takeaways
- You built a bird detector using a pre-trained neural network
- This network has been trained on 1000s of images and can detect a 1000 different types of classes
- You were able to process the image before sending it into the network
- You were able to process the output of the neural network and determine what the network is predicting

## Next Steps
- Make the network even better by training it on additional bird types using transfer learning
- Select an even more powerful network with higher accuarcy
